In [2]:
# 导包
import sys
sys.path.append('..')
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
from tqdm import tqdm
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
# from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
from model import *
warnings.filterwarnings('ignore')

2023-12-08 14:48:19.161689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data_path = '../data_path/'
save_path = '../tmp_results/'
# 做召回评估的一个标志, 如果不进行评估就是直接使用全量数据进行召回
metric_recall = True
user_multi_recall_dict = {}

In [4]:
# 采样数据
# all_click_df = get_all_click_sample(data_path)

# 全量训练集
all_click_df = get_all_click_df(data_path,offline=False)
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
# 对时间戳进行归一化,用于在关联规则的时候计算权重
all_click_df['click_timestamp'] = all_click_df[['click_timestamp']].apply(max_min_scaler)


In [5]:
# item 信息
item_info_df = get_item_info_df(data_path)
item_emb_dict = get_item_emb_dict(data_path,save_path=save_path)

In [6]:
# 由于这里需要做召回评估，所以讲训练集中的最后一次点击都提取了出来
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)

In [6]:
# youtubudnn召回
user_multi_recall_dict['youtubednn_recall'] = youtubednn_u2i_dict(trn_hist_click_df,save_path, topk=20)

#如果报get_item_embedding的错 因为tensorflow版本不兼容的问题  将原来的get_item_embedding拆开就可以了
"""
def get_item_embedding_v2(item_embedding, item_input_layer):
    item_out = tf.gather(item_embedding, item_input_layer)
    return Lambda(lambda x: tf.squeeze(x, 1))(item_out)
"""

100%|██████████| 250000/250000 [00:14<00:00, 16677.46it/s]


4208/4208 [==============================] - 310s 73ms/step - loss: 2.3637


250000it [00:16, 15191.18it/s]


'\ndef get_item_embedding_v2(item_embedding, item_input_layer):\n    item_out = tf.gather(item_embedding, item_input_layer)\n    return Lambda(lambda x: tf.squeeze(x, 1))(item_out)\n'

In [8]:
### 命中率极低
metrics_recall(user_multi_recall_dict['youtubednn_recall'], trn_last_click_df, topk=20)

 topk:  10  :  hit_num:  87 hit_rate:  0.00035 user_num :  250000
 topk:  20  :  hit_num:  158 hit_rate:  0.00063 user_num :  250000


In [8]:
# icf recall
item_type_dict, item_words_dict, item_created_time_dict = get_item_info_dict(item_info_df)

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)

# 文章与文章的相似性矩阵
i2i_sim = itemcf_sim(all_click_df, item_created_time_dict,save_path=save_path)
item_emb_df = pd.read_csv(data_path + '/articles_emb.csv')
# 对于每一篇文章， 基于embedding的相似性返回topk个与其最相似的文章
emb_i2i_sim = embdding_sim(all_click_df, item_emb_df, save_path, topk=10) # topk可以自行设置



# i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))
# emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))

sim_item_topk = 20
recall_item_num = 10
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        item_topk_click, item_created_time_dict, emb_i2i_sim)

user_multi_recall_dict['itemcf_sim_itemcf_recall'] = user_recall_items_dict
pickle.dump(user_multi_recall_dict['itemcf_sim_itemcf_recall'], open(save_path + 'itemcf_recall_dict.pkl', 'wb'))




100%|██████████| 250000/250000 [04:27<00:00, 932.97it/s] 


KeyboardInterrupt: 